In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score

from tqdm import tqdm
import torch
from torch import nn
import torch.nn.functional as F
from scipy.optimize import minimize

/Users/khangtran/anaconda3/envs/torch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Parameters

In [2]:
seed = 2605
folds = 5
num_feat = 16
epoch = 10000
lr = 0.01
alpha = 0.5
mode = 'func'

In [3]:
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

## 2. Utils

In [4]:
def fold_separation(train_df, folds, feat_cols, label):
    skf = StratifiedKFold(n_splits=folds)
    train_df['fold'] = np.zeros(train_df.shape[0])
    for i, (idxT, idxV) in enumerate(skf.split(train_df[feat_cols], train_df[label])):
        train_df.at[idxV, 'fold'] = i

## 3. Read Data

In [5]:
df = pd.read_csv('Data/Bank/formated_bank.csv')
# df['bias'] = np.ones(df.shape[0])
feature_cols = list(df.columns)
feature_cols.remove('y')
feature_cols.remove('z')
feature_cols.remove('label')
feature_cols.remove('is_train')
feature_cols.remove('intercept')
label = 'y'
train_df = df[df['is_train'] == 1].reset_index(drop=True).sample(frac = 1)
test_df = df[df['is_train'] == 0].reset_index(drop=True).sample(frac = 1)
male_df = train_df[train_df['z'] == 1].copy().reset_index(drop=True)
female_df = train_df[train_df['z'] == 0].copy().reset_index(drop=True)
fold_separation(male_df, folds, feature_cols, label)
fold_separation(female_df, folds, feature_cols, label)
train_df = pd.concat([male_df, female_df], axis=0).reset_index(drop=True)

In [6]:
male_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,...,campaign,pdays,previous,poutcome,z,label,intercept,y,is_train,fold
0,60,2.078277,-0.795053,1.371580,0.13576,0.687408,1.052817,0.434409,0.681550,-1.551318,...,-0.236331,-0.47683,-0.37108,-0.47099,1,0,0.0,0,1,0.0
1,65,2.078277,-0.795053,-1.442665,0.13576,-0.363090,1.052817,0.434409,0.681550,0.718653,...,-0.545439,-0.47683,-0.37108,-0.47099,1,1,0.0,1,1,0.0
2,87,2.078277,-0.795053,-0.420689,0.13576,-0.311307,1.052817,0.434409,0.408136,-0.117653,...,-0.545439,-0.47683,-0.37108,-0.47099,1,0,0.0,0,1,0.0
3,61,0.399122,-0.795053,1.371580,0.13576,-0.125285,1.052817,0.434409,0.681550,-0.715013,...,-0.545439,-0.47683,-0.37108,-0.47099,1,1,0.0,1,1,0.0
4,61,2.078277,-0.795053,1.371580,0.13576,1.283076,1.052817,0.434409,0.681550,-0.595541,...,0.072777,-0.47683,-0.37108,-0.47099,1,1,0.0,1,1,0.0


## 4. Normal Logistic Regression (Sklearn)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
clf = LogisticRegression(random_state=0, max_iter=epoch, fit_intercept=False, verbose=1).fit(X=male_df[feature_cols].values, y=male_df[label].values)
clf.score(X=male_df[feature_cols].values, y=male_df[label].values)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           16     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.89229D+02    |proj g|=  4.93350D+03

At iterate   50    f=  1.33082D+02    |proj g|=  3.00376D+01

At iterate  100    f=  1.32400D+02    |proj g|=  2.52900D+00

At iterate  150    f=  1.32330D+02    |proj g|=  2.23351D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   16    181    222      1     0     0   4.614D-02   1.323D+02
  F =   132.32951308656740     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 This problem is unconstrained.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


0.7802197802197802

In [8]:
y_pred = clf.predict_proba(X=male_df[feature_cols].values)[:,1]
log_loss(y_true=male_df[label], y_pred=y_pred)

0.46992917504418436

In [9]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, max_iter=epoch, fit_intercept=False).fit(X=female_df[feature_cols].values, y=female_df[label].values)
clf.score(X=female_df[feature_cols].values, y=female_df[label].values)

0.9362363919129082

In [10]:
y_pred = clf.predict_proba(X=female_df[feature_cols].values)[:,1]
log_loss(y_true=female_df[label], y_pred=y_pred)

0.2336893470970485

In [11]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, max_iter=epoch, fit_intercept=False).fit(X=train_df[feature_cols].values, y=train_df[label].values)
clf.score(X=train_df[feature_cols].values, y=train_df[label].values)

0.9189058339385137

## 5. Logistic Regression with Taylor approximation

In [12]:
feature_cols

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome']

### 5.1 With pytorch

In [13]:
# female
X_fem = female_df[feature_cols].values
y_fem = female_df[label].values.reshape(-1, 1)

X_fem = X_fem/np.linalg.norm(X_fem, ord=2, axis=1).reshape(-1, 1)

# male
X_mal = male_df[feature_cols].values
y_mal = male_df[label].values.reshape(-1, 1)

X_mal = X_mal/np.linalg.norm(X_mal, ord=2, axis=1).reshape(-1, 1)

# train
X = train_df[feature_cols].values
y = train_df[label].values.reshape(-1, 1)

X = X/np.linalg.norm(X, ord=2, axis=1).reshape(-1, 1)

In [14]:
def get_coefficient(X, y, epsilon=None, lbda=None, mode='torch'):
    num_data_point = X.shape[0]
    num_feat = X.shape[1]
    sensitivity = num_feat**2/4 + 3*num_feat
    coff_0 = 1.0
    coff_1 = np.sum(X/2 - X*y, axis = 0).astype(np.float32)
    coff_2 = np.dot(X.T, X).astype(np.float32)
    noise_1 = np.random.laplace(0.0, sensitivity/epsilon, coff_1.shape).astype(np.float32)
    noise_2 = np.random.laplace(0.0, sensitivity/epsilon, coff_2.shape).astype(np.float32)
    if mode == 'scipy':
        return coff_0, (1/num_data_point)*coff_1.reshape(-1, 1), (1/num_data_point)*coff_2
    elif mode == 'scipy_dp':
        coff_1 = coff_1.reshape(-1, 1) 
        coff_1 = coff_1 + noise_1
        coff_2 = coff_2 + np.triu(noise_2, k=0) + np.triu(noise_2, k=1).T + lbda*np.identity(num_feat)
        return coff_0, (1/num_data_point)*coff_1, (1/num_data_point)*coff_2
    elif mode == 'torch':
        return coff_0, (1/num_data_point)*torch.from_numpy(coff_1.reshape(1, -1)), (1/num_data_point)*torch.from_numpy(coff_2)
    elif mode == 'func':
        coff_1 = coff_1 + noise_1
        coff_2 = coff_2 + np.triu(noise_2, k=0) + np.triu(noise_2, k=1).T + lbda*np.identity(num_feat)
        w, V = np.linalg.eig(coff_2)
        indx = np.where(w > 0)[0]
        w = w[indx].astype(np.float32)
        V = V[indx, :].astype(np.float32)
        coff_2 = np.identity(len(w))*w.astype(np.float32)
        coff_1 = np.dot(coff_1, V.T).astype(np.float32)
        return coff_0, (1/num_data_point)*torch.from_numpy(coff_1.reshape(1, -1)), (1/num_data_point)*torch.from_numpy(coff_2), np.linalg.pinv(V)

In [15]:
eps = 1.0
lbd = 10.0

In [16]:
f_coff_0, f_coff_1, f_coff_2, Q_f = get_coefficient(X=X_fem, y=y_fem, epsilon=eps, lbda=lbd, mode=mode)
m_coff_0, m_coff_1, m_coff_2, Q_m = get_coefficient(X=X_mal, y=y_mal, epsilon=eps, lbda=lbd, mode=mode)

In [17]:
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

In [18]:
print(Q_m.shape, Q_f.shape)
print(check_symmetric(m_coff_2),check_symmetric(f_coff_2))

(16, 8) (16, 7)
True True


In [19]:
model_mal = torch.randn((1, m_coff_2.shape[1]), requires_grad=True)
model_fem = torch.randn((1, f_coff_2.shape[1]), requires_grad=True)

In [20]:
tk0 = tqdm(range(epoch), total=epoch)
for ep in tk0:
    loss_m = m_coff_0 + torch.inner(m_coff_1, model_mal) + torch.sum(torch.mul(m_coff_2, torch.inner(model_mal.T, model_mal.T))) # + alpha*torch.norm(model, p=2)
    loss_f = f_coff_0 + torch.inner(f_coff_1, model_fem) + torch.sum(torch.mul(f_coff_2, torch.inner(model_fem.T, model_fem.T)))
    # if ep%500 == 0:
        # print(loss_m.item(), loss_f.item())
    model_mal.retain_grad()
    model_fem.retain_grad()
    loss_m.backward()
    loss_f.backward()
    with torch.no_grad():
        model_mal -= lr * model_mal.grad
        model_fem -= lr * model_fem.grad
        if mode == 'func':
            model_m = np.dot(Q_m, model_mal.numpy().T).T
            model_f = np.dot(Q_f, model_fem.numpy().T).T
            model = torch.from_numpy((model_m + model_f)/2)
            pred_m = torch.sigmoid(torch.inner(torch.from_numpy(X_mal.astype(np.float32)), torch.from_numpy(model_m)))
            pred_f = torch.sigmoid(torch.inner(torch.from_numpy(X_fem.astype(np.float32)), torch.from_numpy(model_f)))
            pred = torch.sigmoid(torch.inner(torch.from_numpy(X.astype(np.float32)), model))
        else:
            model = (model_mal + model_fem)/2
            pred_m = torch.sigmoid(torch.inner(torch.from_numpy(X_mal.astype(np.float32)), model_mal))
            pred_f = torch.sigmoid(torch.inner(torch.from_numpy(X_fem.astype(np.float32)),model_fem))
            pred = torch.sigmoid(torch.inner(torch.from_numpy(X.astype(np.float32)), model))
        pred = pred.numpy()
        pred_m = pred_m.numpy()
        pred_f = pred_f.numpy()

        # print("Train prediction:", pred)
        # print("Male prediction:", pred_m)
        # print("Female prediction:", pred_f)


        acc_mal = accuracy_score(y_true=y_mal, y_pred=np.round(pred_m))
        acc_fem = accuracy_score(y_true=y_fem, y_pred=np.round(pred_f))
        acc = accuracy_score(y_true=y, y_pred=np.round(pred))
        if ep%500 == 0:
            print("Epoch {}: loss male {}, loss female {}, acc male {}, acc female {}, train acc {}".format(ep, loss_m.item(), loss_f.item(), acc_mal, acc_fem, acc))            
    model_mal.grad = torch.zeros(model_mal.size())
    model_fem.grad = torch.zeros(model_fem.size())

  0%|▏                                      | 36/10000 [00:00<00:27, 356.19it/s]

Epoch 0: loss male 12.185113906860352, loss female 1.5745456218719482, acc male 0.7655677655677655, acc female 0.3916537065837221, train acc 0.41636407649479545


  6%|██▏                                   | 591/10000 [00:01<00:20, 461.91it/s]

Epoch 500: loss male 0.5831038951873779, loss female 1.0111029148101807, acc male 0.23443223443223443, acc female 0.6078278900984966, train acc 0.5836359235052045


 11%|███▉                                 | 1058/10000 [00:02<00:19, 460.63it/s]

...

 91%|█████████████████████████████████▋   | 9106/10000 [00:19<00:01, 530.50it/s]

Epoch 9000: loss male 0.5830880403518677, loss female 0.9791527986526489, acc male 0.23443223443223443, acc female 0.6034214618973561, train acc 0.5836359235052045


 95%|███████████████████████████████████▎ | 9533/10000 [00:20<00:00, 529.09it/s]

Epoch 9500: loss male 0.5830880403518677, loss female 0.9791505336761475, acc male 0.23443223443223443, acc female 0.6034214618973561, train acc 0.5836359235052045


100%|████████████████████████████████████| 10000/10000 [00:21<00:00, 463.90it/s]


### 5.1 With scipy

In [21]:
# X = female_df[feature_cols].values
# y = female_df[label].values.reshape(-1, 1)

# X = X/np.linalg.norm(X, ord=2, axis=1).reshape(-1, 1)
# coff_0, coff_1, coff_2 = get_coefficient(X=X, y=y, epsilon=10.0, mode='scipy_dp')
# print(coff_1.shape, coff_2.shape)

In [22]:
# model = np.random.normal(0.0, 1.0, coff_1.shape)
# print(model.shape)

In [23]:
# def sigmoid(x):
#     return np.exp(x)/(1 + np.exp(x))

# def f(w):
#     return coff_0 + np.squeeze(np.dot(coff_1.T, w)) + np.sum(coff_2*np.dot(w, w.T))

In [24]:
# res = minimize(f, model, method='CG')
# print(res)

In [25]:
# best_model = res['x'].reshape(-1,1)
# pred = sigmoid(np.dot(X, best_model))
# print(pred.shape)

In [26]:
# acc = accuracy_score(y_true=y, y_pred=np.round(pred))
# print("Accuracy score: {}".format(acc))

In [27]:
# X = train_df[feature_cols].values
# y = train_df[label].values.reshape(-1, 1)

# X = X/np.linalg.norm(X, ord=2, axis=1).reshape(-1, 1)
# pred = pred = sigmoid(np.dot(X, best_model))
# acc = accuracy_score(y_true=y, y_pred=np.round(pred))
# print("Accuracy score: {}".format(acc))